### Chain

In [12]:
!pip install -q langchain langchain-openai langchain_community google-search-results wikipedia langchain_chroma langchainhub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
elasticsearch 7.17.0 requires urllib3<2,>=1.21.1, but you have urllib3 2.2.3 which is incompatible.


In [ ]:
# import the necessary modules
import getpass
import os

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# provide the OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
# create openai chat model
model = ChatOpenAI(model="gpt-4o")

# create basic string output parser
parser = StrOutputParser()

# create prompt template
system_template = "Translate the following english {text} into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [4]:
# using LCEL to create chain
chain = prompt_template | model | parser

# invoke the chain
chain.invoke({"language": "french", "text": "Hello how are you today ?"})

"Bonjour, comment ça va aujourd'hui ?"

### Agent

In [1]:
# import necessary modules
import getpass
import os

from langchain_openai import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

# provide the OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

# provide the Serp API key
os.environ["SERPAPI_API_KEY"] = getpass.getpass()

In [2]:
# create tools
tools = load_tools(["serpapi"])

# create openai chat model
model = ChatOpenAI(model="gpt-4o")

# initialize agent
agent = initialize_agent(
    tools, 
    model, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

/Users/ruangguru/opt/anaconda3/envs/ds-chatbot-311/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [3]:
# invoke agent
agent("why nvidia stocks are up?")

/Users/ruangguru/opt/anaconda3/envs/ds-chatbot-311/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
To determine why NVIDIA's stock is up, I should look for recent news or events that might have influenced the stock's performance. Let's search for the latest information. 

Action:
```json
{
  "action": "Search",
  "action_input": "NVIDIA stock news October 2023"
}
```

Observation: ['Record quarterly revenue of $35.1 billion, up 17% from Q2 and up 94% from a year ago; Record quarterly Data Center revenue of $30.8 billion, ...', 'The closing price for NVIDIA (NVDA) in October 2023 was $40.77, on October 31, 2023. It was down 7.4% for the month. The latest price is $135.37.', "It really doesn't matter much if you buy Nvidia stock before or after TSMC reports. The stock closed at a new record high on Monday, pushing Nvidia's market ...", 'Discover historical prices for NVDA stock on Yahoo Finance. View daily, weekly or monthly format back to when NVIDIA Corporation stock was issued.', 'Nvidia stock cleared a trendline entry near 139.60 on Oct. 17 

{'input': 'why nvidia stocks are up?',
 'output': "NVIDIA's stock is up due to record quarterly revenue, strong data center performance, increased demand from AI technologies, and a positive outlook from analysts."}

In [4]:
agent("the difference between a stock and a bond")



> Entering new AgentExecutor chain...
Question: What is the difference between a stock and a bond?

Thought: I know that stocks and bonds are both financial instruments but serve different purposes. Let me explain their key differences.

Final Answer: The main difference between a stock and a bond is their nature and the rights they confer to the holder:

1. **Ownership vs. Debt**:
   - **Stock**: Represents ownership in a company. When you buy a stock, you become a shareholder and own a part of the company. You have a claim on part of the company's assets and earnings.
   - **Bond**: Represents a loan made by the investor to a borrower (typically corporate or governmental). When you buy a bond, you are lending money to the issuer in exchange for periodic interest payments and the return of the bond's face value when it matures.

2. **Returns**:
   - **Stock**: Returns come in the form of dividends and capital gains. Dividend payments are not guaranteed and depend on the company's pr

{'input': 'the difference between a stock and a bond',
 'output': "The main difference between a stock and a bond is their nature and the rights they confer to the holder:\n\n1. **Ownership vs. Debt**:\n   - **Stock**: Represents ownership in a company. When you buy a stock, you become a shareholder and own a part of the company. You have a claim on part of the company's assets and earnings.\n   - **Bond**: Represents a loan made by the investor to a borrower (typically corporate or governmental). When you buy a bond, you are lending money to the issuer in exchange for periodic interest payments and the return of the bond's face value when it matures.\n\n2. **Returns**:\n   - **Stock**: Returns come in the form of dividends and capital gains. Dividend payments are not guaranteed and depend on the company's profitability.\n   - **Bond**: Returns are usually fixed and come in the form of regular interest payments (also known as coupon payments). Bonds are generally seen as lower-risk inv

### Use Case : Building RAG Applications

In [ ]:
# import necessary modules
import getpass
import os

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# create utility function to format the documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
# load the document that we want to analyze
# this document is a blog post from Google Research related to react agent
loader = WebBaseLoader(web_paths=("https://research.google/blog/react-synergizing-reasoning-and-acting-in-language-models/",))
docs = loader.load()

# create a text splitter to split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# convert into vector embeddings and save it in the vector store (Chroma)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
# create a retriever from the vector store
retriever = vectorstore.as_retriever()

# we will use rag-prompt from the hub
prompt = hub.pull("rlm/rag-prompt")

# create openai chat model
model = ChatOpenAI(model="gpt-4o")

# create rag-chain model
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [15]:
# invoke the rag-chain
rag_chain.invoke("What is Re-Act Agent?")

'ReAct is a method that integrates reasoning and acting within language models to solve various reasoning and decision-making tasks. It combines reasoning traces and text actions to improve performance, interpretability, and control by enabling language models to interact with and adapt to external environments. This approach systematically outperforms previous methods that focused solely on either reasoning or acting.'

In [16]:
# invoke the rag-chain
rag_chain.invoke("Who is the researcher of react agent?")

'The researchers of the ReAct agent include Shunyu Yao and Yuan Cao, who are affiliated with Google Research, Brain Team.'